# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 3.3

Implement a classifier that achieves an auc, $a$, where $a > 0.7$. This demonstrates that you have discovered some interesting features and tuned some algorithms well. 

See details on how to create the class at the end of the notebook. 

Notes:
1. See details at the end of this notebook.
1. Do not use the target label as features

In [1]:
require './final_project_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

## Question 0 (0 Points)

Include the following files in the next cell. You should have completed them in [Final Project Tests](./final_project_tests.ipynb). You can always go back and add code to these files. Please keep in mind that you have two notebooks that depend on these functions, so make sure they both work.

The next cell should contain ONLY
```ruby
require '/home/<username>/final-project-3/metrics.rb'
require '/home/<username>/final-project-3/decision_trees.rb'
require '/home/<username>/final-project-3/linear_models.rb'
require '/home/<username>/final-project-3/transformers.rb'
```

where ```<username>``` is your username. You can find that in the URL for this page.

In [2]:
require '/home/aidasharif/final-project-3/metrics.rb'
require '/home/aidasharif/final-project-3/decision_trees.rb'
require '/home/aidasharif/final-project-3/linear_models.rb'
require '/home/aidasharif/final-project-3/transformers.rb'

true

In [3]:
require 'digest'
class DownsampleNegatives
  attr_reader :sampling_rate
  def initialize sampling_rate
    @sampling_rate = sampling_rate
  end
  
  def train dataset; end
  
  def update_sampling_rate dataset
    class_counts = Hash.new
    dataset["data"]
      .group_by {|e| e["label"] > 0 ? 1 : 0}
      .each {|k,v| class_counts[k] = v.size}
    @sampling_rate = class_counts[1] / class_counts[0].to_f
  end
  
  def hashprob id
    salt = "eifjcchdivlbreckvgndlvkgdtdjnbcnjldelrgefcgt"
    (Digest::MD5.hexdigest(id.to_s + salt).to_i(16) % 100000).abs / 100000.0
  end
  
  def can_keep? example
    can_keep = true
    can_keep = if example["label"] > 0
      true
    elsif hashprob(example["id"]) < @sampling_rate
      true
    else
      false
    end
    return can_keep
  end

  def apply(example_batch)
    return example_batch.select! {|example| can_keep? example}
  end
end

:apply

In [16]:
class ClassifierThree
  include FinalProjectClassifier
  
  
  def create_training_dataset training_db
    sql = <<SQL
select sk_id_curr, target, name_education_type
, ext_source_1, ext_source_2, ext_source_3, days_birth, days_employed, amt_income_total
, organization_type, amt_annuity, amt_goods_price
 from application_train
order by sk_id_curr
SQL
    dataset = create_dataset training_db, sql    
    dataset["data"].map {|x| x["bias"] = 1}
    
    rng = Random.new SEED
    feature_list = dataset["features"]
    new_dataset = dataset.clone  
    bootstrap=[]
    pos=0
    neg=0

    while pos<12000
      temp=dataset["data"][rng.rand(dataset["data"].size)].clone
      temp["features"]=temp["features"].clone
      if temp["label"]==1.0
        bootstrap << temp
        pos+=1
      end
    end

    while neg<1100
      temp=dataset["data"][rng.rand(dataset["data"].size)].clone
      temp["features"]=temp["features"].clone
      if temp["label"]<1.0
        bootstrap << temp
        neg+=1
      end
    end
    new_dataset["features"]=feature_list
    new_dataset["data"]=bootstrap.shuffle
    
    return new_dataset
  end

  def create_evaluation_dataset evaluation_db
    sql = <<SQL
select sk_id_curr, target, name_education_type
, ext_source_1, ext_source_2, ext_source_3, days_birth, days_employed, amt_income_total
, organization_type, amt_annuity, amt_goods_price
 from application_train
order by sk_id_curr
SQL
    dataset = create_dataset evaluation_db, sql    
    dataset["data"].map {|x| x["bias"] = 1.0}
    
    return dataset

  end

  def create_learners dataset
    linear = LogisticRegressionLearner.new(regularization: 0.01, learning_rate: 0.001, batch_size: 20, epochs: 1)
    randomForest=RandomForestLearner.new(1, num_trees: 80, min_size: 1000, max_depth: 20)
    
    transformer=MeanImputation.new(%w(amt_goods_price amt_annuity ext_source_1 ext_source_2 ext_source_3 days_birth days_employed amt_income_total))
    transformer.train dataset
    transformer.apply dataset["data"]
    
    transformer=AgeRangeAsVector.new()
    transformer.train dataset
    transformer.apply dataset["data"]
    
    transformer=TargetAveraging.new(%w(organization_type))
    transformer.train dataset
    transformer.apply dataset["data"]
    
    transformer=MeanImputation.new(%w(organization_type))
    transformer.train dataset
    transformer.apply dataset["data"]
    
    learners=[randomForest]
    
    return learners
  end

end

:create_learners

## Question 1.1 (5 points)

Validate that the classifier works by trying to create a small dataset and training the model on it. The model will be retrained later. This just verify that the interface is working.

In [17]:
def test_dfd046(test_data)
  test_basics test_data
end

test_data_dfd046 = {classifier: ClassifierThree.new, min_auc: 0.7, max_auc: 1.0, folds: 5}
test_dfd046(test_data_dfd046)

Dev Set:
["name_education_type", "ext_source_1", "ext_source_2", "ext_source_3", "days_birth", "days_employed", "amt_income_total", "organization_type", "amt_annuity", "amt_goods_price"]
{"label"=>1, "id"=>403428, "features"=>{"name_education_type"=>"Higher education", "ext_source_2"=>0.3220169632534332, "ext_source_3"=>0.14825437906109115, "days_birth"=>-10049, "days_employed"=>-1217, "amt_income_total"=>112500, "organization_type"=>"Other", "amt_annuity"=>13725, "amt_goods_price"=>225000}, "bias"=>1}

Evaluation Set:
["name_education_type", "ext_source_1", "ext_source_2", "ext_source_3", "days_birth", "days_employed", "amt_income_total", "organization_type", "amt_annuity", "amt_goods_price"]
{"label"=>1, "id"=>403428, "features"=>{"name_education_type"=>"Higher education", "ext_source_2"=>0.3220169632534332, "ext_source_3"=>0.14825437906109115, "days_birth"=>-10049, "days_employed"=>-1217, "amt_income_total"=>112500, "organization_type"=>"Other", "amt_annuity"=>13725, "amt_goods_pric

## Question 1.2 (15 points)

Measures the 5-fold cross-validation performance of your classifier. This may take a while, so be cognizant of the fact that your classifier may run out of memory--the server is a shared environment.

Notes:

1. This cell will timeout after 3 hours, after which you will see an ```Timeout::Error execution expired``` error. No exceptions.

In [18]:
def test_459bed(test_data)
  run_cross_validation_performance test_data
  test_data[:cross_validation_results]
end
test_459bed(test_data_dfd046)

Creating training dataset
Creating learners
Running 5-fold cross validation
5-fold CV: RandomForestLearner, parameters: {"num_trees"=>80, "min_size"=>1000, "max_depth"=>20}
0.7381115211839754
{
  "RandomForestLearner": {
    "learner": "RandomForestLearner",
    "parameters": {
      "num_trees": 80,
      "min_size": 1000,
      "max_depth": 20
    },
    "folds": 5,
    "mean_train_metric": 0.8234235482626792,
    "stdev_train_metric": 0.0017673205387523166,
    "mean_test_metric": 0.7381115211839754,
    "stdev_test_metric": 0.009932164043502355
  }
}


,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,RandomForestLearner,"{""num_trees""=>80, ""min_size""=>1000, ""max_depth""=>20}",5,0.8234235482626792,0.0017673205387523166,0.7381115211839754,0.009932164043502355


In [19]:
def test_afdef8(test_data)
  test_cross_validation_performance test_data
end
test_afdef8(test_data_dfd046)

Best Learner
RandomForestLearner
AUC = 0.7381115211839754


## Question 1.3 (20 points)

Plots the ROC curve on the dev dataset, which is a separate database. Checks that the AUC is within target. Note that this assumes the model has already been trained.

In [20]:
def test_bf94da(test_data)
  test_data[:db] = dev_db()
  test_data[:db_size] = 15334
  test_data[:name] = "dev_eval"
  test_evaluation_set_performance test_data
end
test_bf94da(test_data_dfd046)

Creating evaluation dataset
Evaluating classifier
[[100002, 0.9627516210417719], [100007, 0.9265320337555462], [100015, 0.8213065300306626], [100025, 0.910729919679287], [100068, 0.9174975553386151]]
Validating predictions against labels from database
Plotting ROC curve
dev_eval set AUC: 0.7047371664034525


#<Nyaplot::Frame:0x000000000b81ad20 @properties={:panes=>[#<Nyaplot::Plot:0x0000000021c7e8d8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000000000b81b8b0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :title=>"AUC: 0.7047"}, :data=>"981f6309-5954-4b62-b1d0-284e7a8be720"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"False Positive Rate", :y_label=>"True Positive Rate", :legend=>true, :zoom=>true, :width=>800, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>], :data=>{"981f6309-5954-4b62-b1d0-284e7a8be720"=>#<Nyaplot::DataFrame:0x000000000c0df230 @name="981f6309-5954-4b62-b1d0-284e7a8be720", @rows=[{:x=>1.0, :y=>1.0}, {:x=>0.9999290830437557, :y=>1.0}, {:x=>0.9998581660875115, :y=>1.0}, {:x=>0.9997872491312673, :y=>1.0}, {:x=>0.9997163321750231, :y=>1.0}, {:x=>0.9996454152187788, :y=>1.0}, {:x=>0.9995744982625345, :y=>1.0}, {:x=>0.9995035813062904, :y=>1.0}, {:x=>0.9994326643500461, :y=>1.0}, {:x=>0.9993617473938019, :y=>1.0}, {:x=>0.9992908304375576, :y=>1.0}, {:x=>0.9992199134813133, :y=>1.0}, {:x=>0.9991489965250692, :y=>1.0}, {:x=>0.9990780795688249, :y=>1.0}, {:x=>0.9990071626125807, :y=>1.0}, {:x=>0.9989362456563364, :y=>1.0}, {:x=>0.9988653287000921, :y=>1.0}, {:x=>0.998794411743848, :y=>1.0}, {:x=>0.9987234947876037, :y=>1.0}, {:x=>0.9986525778313595, :y=>1.0}, {:x=>0.9985816608751152, :y=>1.0}, {:x=>0.998510743918871, :y=>1.0}, {:x=>0.9984398269626268, :y=>1.0}, {:x=>0.9983689100063825, :y=>1.0}, {:x=>0.9982979930501383, :y=>1.0}, {:x=>0.998227076093894, :y=>1.0}, {:x=>0.9981561591376498, :y=>1.0}, {:x=>0.9980852421814056, :y=>1.0}, {:x=>0.9980143252251613, :y=>1.0}, {:x=>0.9979434082689171, :y=>1.0}, {:x=>0.9978724913126729, :y=>1.0}, {:x=>0.9978015743564286, :y=>1.0}, {:x=>0.9977306574001844, :y=>1.0}, {:x=>0.9976597404439401, :y=>1.0}, {:x=>0.997588823487696, :y=>1.0}, {:x=>0.9975179065314517, :y=>1.0}, {:x=>0.9974469895752074, :y=>1.0}, {:x=>0.9973760726189632, :y=>1.0}, {:x=>0.9973051556627189, :y=>1.0}, {:x=>0.9972342387064748, :y=>1.0}, {:x=>0.9971633217502305, :y=>1.0}, {:x=>0.9970924047939862, :y=>1.0}, {:x=>0.997021487837742, :y=>1.0}, {:x=>0.9969505708814977, :y=>1.0}, {:x=>0.9968796539252536, :y=>1.0}, {:x=>0.9968087369690093, :y=>1.0}, {:x=>0.996737820012765, :y=>1.0}, {:x=>0.9966669030565208, :y=>1.0}, {:x=>0.9965959861002766, :y=>1.0}, {:x=>0.9965250691440324, :y=>1.0}, {:x=>0.9964541521877881, :y=>1.0}, {:x=>0.9963832352315438, :y=>1.0}, {:x=>0.9963123182752996, :y=>1.0}, {:x=>0.9962414013190554, :y=>1.0}, {:x=>0.9961704843628112, :y=>1.0}, {:x=>0.9960995674065669, :y=>1.0}, {:x=>0.9960286504503226, :y=>1.0}, {:x=>0.9959577334940785, :y=>1.0}, {:x=>0.9958868165378342, :y=>1.0}, {:x=>0.99581589958159, :y=>1.0}, {:x=>0.9957449826253457, :y=>1.0}, {:x=>0.9956740656691014, :y=>1.0}, {:x=>0.9956031487128573, :y=>1.0}, {:x=>0.995532231756613, :y=>1.0}, {:x=>0.9954613148003688, :y=>1.0}, {:x=>0.9953903978441245, :y=>1.0}, {:x=>0.9953194808878802, :y=>1.0}, {:x=>0.9952485639316361, :y=>1.0}, {:x=>0.9951776469753918, :y=>1.0}, {:x=>0.9951067300191476, :y=>1.0}, {:x=>0.9950358130629033, :y=>1.0}, {:x=>0.9949648961066591, :y=>1.0}, {:x=>0.9948939791504149, :y=>1.0}, {:x=>0.9948230621941706, :y=>1.0}, {:x=>0.9947521452379264, :y=>1.0}, {:x=>0.9946812282816822, :y=>1.0}, {:x=>0.9946103113254379, :y=>1.0}, {:x=>0.9945393943691937, :y=>1.0}, {:x=>0.9944684774129494, :y=>1.0}, {:x=>0.9943975604567052, :y=>1.0}, {:x=>0.994326643500461, :y=>1.0}, {:x=>0.9942557265442167, :y=>1.0}, {:x=>0.9941848095879725, :y=>1.0}, {:x=>0.9941138926317282, :y=>1.0}, {:x=>0.994042975675484, :y=>1.0}, {:x=>0.9939720587192398, :y=>1.0}, {:x=>0.9939011417629955, :y=>1.0}, {:x=>0.9938302248067513, :y=>1.0}, {:x=>0.993759307850507, :y=>1.0}, {:x=>0.9936883908942629, :y=>1.0}, {:x=>0.9936174739380186, :y=>1.0}, {:x=>0.9935465569817743, :y=>1.0}, {:x=>0.9934756400255301, :y=>1.0}, {:x=>0.9934047230692858, :y=>1.0}, {:x=>0.9933338061130417, :y=>1.0}, {:x=>0.9932628891567974, :y=>1.0}, {:x=>0.9931919722005531, :y=>1.0}, {:x=

## Question 1.3 (60 Points)

Tests your model on the test dataset. This is a hidden test, so you will not see this until after you submit. 

In [21]:
def test_7f956a(test_data)
  hidden_test_data = test_data.clone
end
test_7f956a(test_data_dfd046)
test_data_dfd046 = nil

## Notes

We will use all the models, feature extraction, and evaluation methods you learned in this course on a realistic dataset. In the final project, you will create better and better classifiers for the final project dataset. In each set of classifiers, create a dataset directly from the database. Assume that the database will be provided to you and **do not assume** that the database will be the same all the time.

Note that the test set performance will be a **HIDDEN** test. You may find that your model does not perform as well on the hidden test as on the training set. To mitigate this, we are using cross-validation and a hold-out set which is about the same size as the testing set. 

Note the following requirements:
1. Focus on data preparation and creating, normalizing, understanding new features.
1. You may use R notebooks on this server for exploration but anything you obtain there must run in this notebook
1. Use your own implementation of models, which you will have refactored as part of [Final Project Tests](./final_project_tests.ipynb)
1. Avoid creating new models that we did not cover in the class. There is no credit for fancy models.
1. Do not use the target label, or anything that is derived based on the training label as features. 
1. You may talk to other students about your solution, but do not share code. This is an individual assignment.


For this and all subsequent questions, create your own class following the ```FinalProjectClassifier``` interface below.

Each example in the dataset you create will have an ```id``` field. This is how we will track performance. Please note that that you must provide a score for each example in the evaluation database provided to your classifier. So please take care to keep all examples in the evaluation set. You may sample the training set as much as you want. 

The scores will extracted from the database and your ```AUCMetric``` will be applied.

Here is an example of a classifier:
```ruby
class ClassifierOne
  include FinalProjectClassifier
  
  # Run SQL on the database and any transforms needed for the training database
  def create_training_dataset training_db
    sql = <<SQL
select sk_id_curr, target
from application_train  
order by sk_id_curr
SQL
    dataset = create_dataset training_db, sql    
    dataset["data"].map {|x| x["bias"] = 1.0}
    return dataset
  end
  
  # Run SQL on the database and any transforms needed for the evaluation database
  def create_evaluation_dataset evaluation_db
    return create_training_dataset evaluation_db
  end
  
  #Return an array of learners
  def create_learners dataset
    linear = LogisticRegressionLearner.new(regularization: 0.01, learning_rate: 0.001, batch_size: 20, epochs: 1)
    return [linear]
  end

end
```

